In [21]:
import numpy as np
from typing import *

class NODE:
    def __init__(self, point: List, level: int, left: None = None, right: None=None):
        self.point = point
        self.level = level
        self.left = left
        self.right = right

def level_travel(root: NODE) -> List:
    nn = root
    if nn is None:
        return None
    ans = []
    res = [nn]
    while len(res):
        tmp = []
        for node in res:
            tmp.append(node)
        ans.append([i.point for i in tmp])
        res.clear()
        for node in tmp:
            if node.left:
                res.append(node.left)
            if node.right:
                res.append(node.right)
    return ans
    

In [ ]:
X=np.array([1,3])
root=NODE(X,0)
root.left=NODE(X+np.array([1,1]),1)
root.right=NODE(X,1)
root.left.right=NODE(X+np.array([1,1]),1)
root.right.left=NODE(X+np.array([1,1]),1)

ans=level_travel(root)
for i in ans:
    print(i)


In [ ]:
root=NODE(1,0)
root.left=NODE(2,1)
root.right=NODE(3,1)
ans=level_travel(root)
for i in ans:
    print(i)


In [22]:
from typing import *

def get_kd_tree(X: List, level: int) -> NODE:
    """ 
    X : m 维特征向量 
    level : 以第 level 维进行超平面划分
    """
    n = len(X) # 向量数
    if n==0:
        return None
    
    dim = len(X[0]) # 特征维数
    level = level%dim # 第level维进行划分
    X = sorted(X, key=lambda x: x[level]) # 按照第level维进行排序
    mid = n//2
    root = NODE(X[mid], level) # 中值
    root.left = get_kd_tree(X[:mid], level+1)
    root.right = get_kd_tree(X[mid+1:], level+1)
    return root


X=[[2,3],[5,7],[9,6],[4,5],[6,4],[7,2]]
root = get_kd_tree(X, 0)
print(root)
ans = level_travel(root)
for i in ans:
    print(i)


[[6, 4]]
[[4, 5], [9, 6]]
[[2, 3], [5, 7], [7, 2]]


In [48]:
import numpy as np

# 创建一个二维NumPy数组
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 定义你想要移除的行的索引
row_to_remove = 1  # 移除第二行

# 创建一个新的数组，不包含指定的行
new_arr = np.delete(arr, row_to_remove, axis=0)

print(new_arr)


[[1 2 3]
 [7 8 9]]


In [ ]:
x[1:]


In [ ]:
def kd_create_iterative(X: np.ndarray):
    stack = [(X, 0)]  # 初始栈，包含特征向量X和初始level
    root = None  # 根节点初始化为None

    while stack:
        X, level = stack.pop()
        n = len(X)
        dim = X.shape[1]
        level = level % dim
        idx = np.argsort(X[:, level])
        X = X[idx]
        node = NODE(X[n//2], level)

        if root is None:
            root = node
        else:
            current = root
            while True:
                if node.point[level] < current.point[level]:
                    if current.left is None:
                        current.left = node
                        break
                    else:
                        current = current.left
                else:
                    if current.right is None:
                        current.right = node
                        break
                    else:
                        current = current.right

        X_left, X_right = np.split(X, [n//2])
        if len(X_left):
            stack.append((X_left, level+1))
        if len(X_right):
            stack.append((X_right, level+1))

    return root


In [ ]:
X=np.array([[2,3],[5,7],[9,6],[4,5],[6,4],[7,2]])
root = kd_create(X, 0)
print(root)
print("3333")


In [ ]:
x_idx = np.argsort(X[:, 1])
X = X[x_idx]
X = X[:1]
X


In [ ]:
import numpy as np
from scipy.spatial import cKDTree

# 假设我们有以下规则的二维 NumPy 数组
# 每一行代表空间中的一个点，这里假设是二维空间
points = np.array([
    [1, 2],
    [3, 4],       
    [5, 6],
    [7, 8]
])

# 使用 points 构造 KD 树
tree = cKDTree(points)

# 现在你可以使用 KD 树进行查询，例如查找最近的点
point_to_query = [1.5, 2.5]
distance, index = tree.query(point_to_query)
print(f"最近的点是 points[{index}]，距离是 {distance}")
